http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=1&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate=

In [1]:
import time
import os
import json
from datetime import datetime
import math

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from tqdm import tqdm_notebook

In [24]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm_notebook

url page 2개  
'http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=%s&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate='%(str(idx))

In [25]:
for idx in range(1, 3):
    print('http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=%s&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate='%(str(idx)))

http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=1&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate=
http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=2&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate=


In [30]:
def get_qnaList(url):
#     url = 'http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=1&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate='

    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    qna_list = soup.select('td.td_tit > a')

    for idx, qna in enumerate(qna_list):
        qna_list[idx] = 'http://www.fss.or.kr/edu/notice/faq/' + qna['href']

    return qna_list

In [31]:
def get_qnaContent(url):
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    qna = dict()
    qna['제목'] = soup.select_one('div.tblWrap > table > tbody > tr:nth-of-type(1) > td').text
    qna['등록일'] = soup.select('div.tblWrap > table > tbody > tr:nth-of-type(2) > td')[0].text
    qna['조회수'] = soup.select('div.tblWrap > table > tbody > tr:nth-of-type(2) > td')[1].text
    try:
        qna['첨부파일'] = soup.select_one('div.tblWrap > table > tbody > tr:nth-of-type(3) > td').text
    except:
        qna['첨부파일'] = 'NULL'
    qna['답변'] = soup.select_one('div.tblWrap > table > tbody > tr:nth-of-type(4) > td').text
    qna['담당부서'] = soup.select_one('div.contWrap > div.context > div.tblWrap > table > tbody > tr:nth-of-type(5) > td').text
    qna['담당팀'] = soup.select_one('div.contWrap > div.context > div.tblWrap > table > tbody > tr:nth-of-type(5) > td:nth-of-type(2)').text
    qna['자료문의'] = soup.select_one('div.contWrap > div.context > div.tblWrap > table > tbody > tr:nth-of-type(6) > td').text
    
    
    return qna

In [32]:
def main():
    total_qnaList = list()
    fileName = '금융감독원(금융교육센터)'
    csv_file = open(str(fileName) + '.csv', "w", encoding = 'utf-8-sig', newline = '')
    
    columns = ['제목', '등록일', '조회수', '첨부파일', '답변', '담당부서', '담당팀', '자료문의']
    
    wr = csv.writer(csv_file)    
    wr.writerow(columns)
    
    for idx in range(1, 3):
        url = 'http://www.fss.or.kr/edu/notice/faq/faq_list.jsp?edubbs=all&page=%s&s_title=&s_kind=&sdate=&b_skincode=1327884662812&menu=&edate=&cate='%(str(idx))
        qnaList = get_qnaList(url)
        
        for qna in qnaList:
            qnaContent = get_qnaContent(qna)
            print(qnaContent)
            total_qnaList.append(qnaContent)
    
    
    # '모든 수집 복지서비스'에 대한 반복문 -> 복지 서비스 하나
    for info_dict in total_qnaList:
        # 'columns'을 돌면서 info_dict가 해당 col을 가지고 있으면 정보를 적고, 없으면 NULL
        wr.writerow([info_dict.get(col, 'NULL') for col in columns])
    
    print('==================== 저장 완료 =======================')
    csv_file.close()
            
    

In [33]:
if __name__ == "__main__":
    main()

{'제목': '금융공모전 접수시 "적절한 파일이 아닙니다"라는 경고창이 표시되면서 접수가 되지 않습니다.', '등록일': '2019-10-01', '조회수': '192', '첨부파일': '\n', '답변': '\n\xa0\n첨부하신 파일의 파일명이 아래 사항에 해당되는 경우 파일명을 수정하신 후\xa0다시 시도하시기 바랍니다.\n\xa0\n1. 파일명에서 확장자명 앞에 점(.) 등이 중복 입력\xa0→ 파일명 수정\n\xa0\xa0\xa0\xa0(ex.\xa0 금융창작물_학교장추천서..pdf\xa0\xa0 →\xa0 금융창작물_학교장추천서.pdf)\n\xa0\n2. 파일명 확장자가 대문자로 입력 → 소문자로 변경\n\xa0\xa0\xa0 (ex. PDF → pdf)\n\xa0\n3. 파일명에 특수문자 포함\xa0 → 특수문자 삭제\n\xa0\xa0\xa0 (ex. 금융창작물&학교장추천서.jpg\xa0 → 금융창작물 학교장추천서.jpg)\n\xa0\n', '담당부서': '금융교육국', '담당팀': '금융교육기획팀', '자료문의': '02-3145-5983'}
{'제목': '금융공모전 작품을 제출하려고 하는데, 이메일 인증이 안됩니다.', '등록일': '2019-10-01', '조회수': '165', '첨부파일': '\n', '답변': '\n\xa0\n이메일 주소를 입력하신 후, 도메인 주소(@naver.com, @hanmail.net 등)를 \n드롭다운 목록에서 선택하지 않고 직접 입력하신 경우에도\n드롭다운 목록의 가장 하단에 있는 "직접 입력" 이라는 항목을 선택해주셔야 합니다.\n\xa0\n', '담당부서': '금융교육국', '담당팀': '금융교육기획팀', '자료문의': '02-3145-5983'}
{'제목': 'FSS금융아카데미는 언제 시행되나요?', '등록일': '2019-05-02', '조회수': '530', '첨부파일': '\n', '답변': '\nFSS 금융아카데미의 경우,\xa0매년 3-4월부터 12월까지 매달 마지막주 금요일에 일반과정을 개최하